# Загрузка данных анкет и проведение МАИ, анализ полученных результатов
Данные: заполненные анкеты экспертов (таблицы Excel) 

In [1]:
import re
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib.pyplot as plt
from random import randint

In [2]:
df = pd.read_excel(r'C:\Users\user\инциденты\Экс3.xlsx')
df.head()

,Тип нарушения,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала
0,Документация,1.00,5.000000,5.000000,4.00,5.0,5.0,5
1,Экспертиза промышленной безопасности,0.20,1.000000,5.000000,4.00,6.0,6.0,5
2,Готовность к ликвидации последствий аварий и и...,0.20,0.200000,1.000000,4.00,7.0,6.0,7
3,Оформление информационными знаками и знаками б...,0.25,0.250000,0.250000,1.00,4.0,4.0,5
4,Содержание объектов и оборудования,0.20,0.166667,0.142857,0.25,1.0,7.0,8


In [3]:
print('Число строк и столбцов в наборе данных:\n', df.shape)

Число строк и столбцов в наборе данных:
 (7, 8)


In [4]:
# типы столбцов 
df.dtypes

Тип нарушения                                                 object
Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала         int64
dtype: object

In [5]:
#удаление столбца типа object для дальнейшей работы с фреймом 
df.pop('Тип нарушения') 
print(df)

   Документация   Экспертиза промышленной безопасности  \
0           1.00                              5.000000   
1           0.20                              1.000000   
2           0.20                              0.200000   
3           0.25                              0.250000   
4           0.20                              0.166667   
5           0.20                              0.166667   
6           0.20                              0.200000   

   Готовность к ликвидации последствий аварий и инцидентов  \
0                                           5.000000         
1                                           5.000000         
2                                           1.000000         
3                                           0.250000         
4                                           0.142857         
5                                           0.166667         
6                                           0.142857         

   Оформление информационными знаками 

In [6]:
# типы столбцов 
df.dtypes

Документация                                                 float64
Экспертиза промышленной безопасности                         float64
Готовность к ликвидации последствий аварий и инцидентов      float64
Оформление информационными знаками и знаками безопасности    float64
Содержание объектов и оборудования                           float64
Охранные зоны (ОЗ)                                           float64
Организация рабочего процесса и безопасности персонала         int64
dtype: object

In [7]:
# также меням тип данных Документации
df['Документация '] = df['Документация '].astype(float)

In [8]:
df.loc[:, df.columns[df.dtypes == 'float64']].head(7)

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ)
0,1.00,5.000000,5.000000,4.00,5.000000,5.000000
1,0.20,1.000000,5.000000,4.00,6.000000,6.000000
2,0.20,0.200000,1.000000,4.00,7.000000,6.000000
3,0.25,0.250000,0.250000,1.00,4.000000,4.000000
4,0.20,0.166667,0.142857,0.25,1.000000,7.000000
5,0.20,0.166667,0.166667,0.25,0.142857,1.000000
6,0.20,0.200000,0.142857,0.20,0.125000,0.111111


In [9]:
DF_1=df[:].copy()

In [10]:
#Найдем сумму всех столбцов, для того чтобы в дальнейшем пересчитать полученные значения
df.sum ()

Документация                                                  2.250000
Экспертиза промышленной безопасности                          6.983333
Готовность к ликвидации последствий аварий и инцидентов      11.702381
Оформление информационными знаками и знаками безопасности    13.700000
Содержание объектов и оборудования                           23.267857
Охранные зоны (ОЗ)                                           29.111111
Организация рабочего процесса и безопасности персонала       40.000000
dtype: float64

In [11]:
# Считаем значения для нашей матрицы, путем деления каждого значения в столбце на сумму столбца
df['new_Документация'] = df['Документация '].astype(float).div(df['Документация '].sum())
df['new_Экспертиза'] = df['Экспертиза промышленной безопасности'].astype(float).div(df['Экспертиза промышленной безопасности'].sum())
df['new_Готовность'] = df['Готовность к ликвидации последствий аварий и инцидентов'].astype(float).div(df['Готовность к ликвидации последствий аварий и инцидентов'].sum())
df['new_Оформление'] = df['Оформление информационными знаками и знаками безопасности'].astype(float).div(df['Оформление информационными знаками и знаками безопасности'].sum())
df['new_Содержание'] = df['Содержание объектов и оборудования '].astype(float).div(df['Содержание объектов и оборудования '].sum())
df['new_ОЗ'] = df['Охранные зоны (ОЗ)'].astype(float).div(df['Охранные зоны (ОЗ)'].sum())
df['new_Организация'] = df['Организация рабочего процесса и безопасности персонала'].astype(float).div(df['Организация рабочего процесса и безопасности персонала'].sum())

In [12]:
df.head()

,Документация,Экспертиза промышленной безопасности,Готовность к ликвидации последствий аварий и инцидентов,Оформление информационными знаками и знаками безопасности,Содержание объектов и оборудования,Охранные зоны (ОЗ),Организация рабочего процесса и безопасности персонала,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,1.00,5.000000,5.000000,4.00,5.0,5.0,5,0.444444,0.715990,0.427263,0.291971,0.214889,0.171756,0.125
1,0.20,1.000000,5.000000,4.00,6.0,6.0,5,0.088889,0.143198,0.427263,0.291971,0.257866,0.206107,0.125
2,0.20,0.200000,1.000000,4.00,7.0,6.0,7,0.088889,0.028640,0.085453,0.291971,0.300844,0.206107,0.175
3,0.25,0.250000,0.250000,1.00,4.0,4.0,5,0.111111,0.035800,0.021363,0.072993,0.171911,0.137405,0.125
4,0.20,0.166667,0.142857,0.25,1.0,7.0,8,0.088889,0.023866,0.012208,0.018248,0.042978,0.240458,0.200


In [13]:
# Получили новую таблицу теперь отделим значения нашей матрицы от основной таблицы
df_new = df[['new_Документация', 'new_Экспертиза', 'new_Готовность', 'new_Оформление', 'new_Содержание', 'new_ОЗ', 'new_Организация']]
df_new

,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация
0,0.444444,0.715990,0.427263,0.291971,0.214889,0.171756,0.125
1,0.088889,0.143198,0.427263,0.291971,0.257866,0.206107,0.125
2,0.088889,0.028640,0.085453,0.291971,0.300844,0.206107,0.175
3,0.111111,0.035800,0.021363,0.072993,0.171911,0.137405,0.125
4,0.088889,0.023866,0.012208,0.018248,0.042978,0.240458,0.200
5,0.088889,0.023866,0.014242,0.018248,0.006140,0.034351,0.225
6,0.088889,0.028640,0.012208,0.014599,0.005372,0.003817,0.025


In [14]:
#Вычислим средние значение по строке и запишем полученный результот в mean
df_new['mean'] = df_new.mean(axis=1)
df_new.head()

C:\Users\user\anaconda3\envs\envpy3.5\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,new_Документация,new_Экспертиза,new_Готовность,new_Оформление,new_Содержание,new_ОЗ,new_Организация,mean
0,0.444444,0.715990,0.427263,0.291971,0.214889,0.171756,0.125,0.341616
1,0.088889,0.143198,0.427263,0.291971,0.257866,0.206107,0.125,0.220042
2,0.088889,0.028640,0.085453,0.291971,0.300844,0.206107,0.175,0.168129
3,0.111111,0.035800,0.021363,0.072993,0.171911,0.137405,0.125,0.096512
4,0.088889,0.023866,0.012208,0.018248,0.042978,0.240458,0.200,0.089521


In [15]:
#проверка, среднее значение в сумме должно быть равно 1
df_new['mean'].sum()

1.0000000000000002

In [16]:
# Далее проведем проверку на согласованность, 
#для этого создадим из нашей полученной выше таблицы матрицы для дальнейшего их перемножения
df_new8 =df_new['mean']

# удаляем из нашей таблицы значениея столбцы mean
#df_new.pop('mean')

df_new1 =DF_1.loc[0]

df_new2 =DF_1.loc[1]

df_new3 =DF_1.loc[2]

df_new4 =DF_1.loc[3]

df_new5 =DF_1.loc[4]

df_new6 =DF_1.loc[5]

df_new7 =DF_1.loc[6]

In [17]:
#Проверка на согласованность
res1 = np.dot(df_new1,df_new8)
print(res1)
res2 = np.dot(df_new2,df_new8)
print(res2)
res3 = np.dot(df_new3,df_new8)
print(res3)
res4 = np.dot(df_new4,df_new8)
print(res4)
res5 = np.dot(df_new5,df_new8)
print(res5)
res6 = np.dot(df_new6,df_new8)
print(res6)
res7 = np.dot(df_new7,df_new8)
print(res7)

3.537023348736289
2.531759601973282
1.8237375832891503
0.9992657171157886
0.8574275515117553
0.4581420119714397
0.19886555212004875


In [18]:
sum_new = res1 + res2 + res3 + res4 + res5 + res6 + res7
print(sum_new)

10.406221366717753


In [19]:
# Вычислем значения CI, RI, CR
CI = (sum_new - 7)/(7-1)
print(CI)
RI = (1.98*(7-2))/7
print(RI)
CR = CI/RI
print(CR)

0.5677035611196256
1.4142857142857144
0.401406558367412


In [20]:
# Получилось, что значение CR=0.4, что больше 0,1 значит матрицу нельзя считать согласованной
# Проделаем аналогичные действия с другими экспертами